In [ ]:
import numpy as np 
import pandas as pd 
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
def create_CPM(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: create_CPM(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)#считаем CPM
df.drop(['total_revenue'], axis = 1, inplace=True) # Удаляем зависимую переменую
df = df[df['CPM'].between(df['CPM'].quantile(.00), df['CPM'].quantile(.95))] # Обрезаем по 95 квантилю
df = df[df['CPM'] >= 0] # Берем положительные значения CPM

In [ ]:
train = df[df['date'] < '2019-06-22']  # Делим траин/тест
test = df[df['date'] >= '2019-06-22']

x_train, y_train = train.drop(columns = ['CPM', 'date']), train['CPM']
x_test, y_test = test.drop(columns = ['CPM', 'date']), test['CPM']

In [ ]:
model = CatBoostRegressor()
model.fit(x_train, y_train)

pred = model.predict(x_test)

In [ ]:
print("MSE: ", mean_squared_error(pred, y_test))
#MSE:  2698.1483480223133